In [57]:
import sys
sys.path.append("..")

In [88]:
import pandas as pd 
df = pd.read_csv( "/mnt/d/mine2sirius_pipe/source/tests/out/formula_identifications.tsv" , sep="\t" )
assert df.loc[0]["molecularFormula"] == "C30H61NO4S"